In [1]:
import pandas as pd

folder = 'H:\\My Drive\\PROJECTS\\PSI 2022-2025\\XRF fundamentals vs. MVA'

In [3]:
sens_df = pd.read_csv(folder+'\\sensitivities.csv')

info = pd.read_csv(folder+'\\data\\dataset_summary.csv')
element_list = info.element.values

In [7]:
# using PLS because best %RMSEP results
model = 'PLS'
# using both filters bc lowest errors
filt = 'both'

for filt in [1,2]:
    for element in element_list:
        coeff = pd.read_csv(f'{folder}\\models\\{filt}\\{element}_{model}_coefs.csv')
        vector = pow(coeff, 2).sum().pow(.5)  #square root of sum of squares
        loq = 10 * vector * #sens#

,element,n_train,n_test,units,median,mean,stdev
0,SiO2,1283,321,wt%,26.00,33.45,25.79
1,TiO2,493,123,wt%,0.86,0.98,0.78
2,Al2O3,1512,378,wt%,13.94,13.52,3.48
3,Fe2O3,389,97,wt%,5.41,6.35,4.55
4,MgO,405,101,wt%,1.80,3.26,3.31
5,MnO,411,103,wt%,0.09,0.11,0.08
6,CaO,427,107,wt%,3.95,6.07,5.51
7,P2O5,352,88,wt%,0.12,0.13,0.11
8,As,62,16,ppm,0.50,0.98,1.04
9,Bi,29,7,ppm,0.09,0.12,0.11


In [ ]:
def get_lbdq(folder, file_list):
    
    print("LBDQ:")
    
    for filter_n in ['O1', 'O2']:
        
        # get sensitivity value
        sensitivity = o1_sens if filter_n == 'O1' else o2_sens
        
        coeffs = []
        ftype = filter_n + "_coeff"

        # read models
        for file in tqdm(file_list):
            if ftype in file:       
                path = folder + file
                data = pd.read_csv(path, skiprows = [0])
                coeffs.append(data)

        # convert to dataframe
        coeffs = pd.concat(coeffs).T

        # adapt to different element naming b/w datasets
        coeffs.columns = coeffs.iloc[0].map(lambda x: x.split()[0])

        coeffs = coeffs.drop(coeffs.index[0])

        # calculate regression vectors
        vector_list = coeffs.pow(2).sum().pow(.5)  #square root of sum of squares

        # populate lists
        elem_list = coeffs.columns
        filt_list = list(filter_n * len(elem_list))

        df = pd.DataFrame({'element' : elem_list,
                           'vector' : vector_list,
                           'filter' : filt_list
        }).reset_index(drop = True)

        # calculate values
        types = ['LOB', 'LOD', 'LOQ']
        factors = [1.645, 3.3, 10]

        for i in range(len(types)):
            df[types[i]] = factors[i] * sensitivity * df['vector']

        # change col formats
        cols = df.columns.drop('element')
        df[cols] = df[cols].apply(pd.to_numeric)
        df['element'] = df['element'].astype(str)

        return df

In [ ]:
def get_results(regression, n_range):
    
    print('Calculating for', regression, n_range)

    folder = fp+"\\models\\"+regression+"\\"+n_range+"\\"
    file_list = os.listdir(folder)

    # calculate lbdq
    lbdq = get_lbdq(folder, file_list)
    # calculate rmsep with lbdq results
    rmsep = get_rmsep(folder, file_list, comps, lbdq)
    # merge results
    df = pd.merge(lbdq, rmsep, how='outer', on='element')
    df.insert(loc=2, column='num_range', value=n_range)
    # return full results
    return df 